A) Basic Entailment Using simple Rule Based Method 

In [3]:
# Step 1: Import Required Libraries

import pandas as pd

import nltk

from sklearn.metrics import accuracy_score



# Download necessary resources for NLTK

nltk.download('punkt')



# Step 2: Load the Dataset

# Assume the dataset is in the form of a CSV file containing two sentences and a label ('entailment' or 'non-entailment')

# Replace 'text_entailment_dataset.csv' with the path to your dataset

data = pd.read_csv('/kaggle/input/textual-entailment-dataset/validation.csv')



# Step 3: Preprocess the Data

# Function to preprocess text by tokenizing and converting to lowercase

def preprocess(text):

    return nltk.word_tokenize(text.lower())



# Apply preprocessing to both sentences in each pair

data['sentence1_tokens'] = data['text1'].apply(preprocess)

data['sentence2_tokens'] = data['text2'].apply(preprocess)



# Step 4: Define Simple Rule-Based Method

# Function to check if all tokens in sentence2 are in sentence1

def simple_rule_based_entailment(s1, s2):

    return set(s2).issubset(set(s1))



# Apply the rule-based method to each sentence pair

data['prediction'] = data.apply(lambda row: simple_rule_based_entailment(row['sentence1_tokens'], row['sentence2_tokens']), axis=1)



# Step 5: Evaluate the Model

# Calculate the accuracy of the predictions

accuracy = accuracy_score(data['label'], data['prediction'])

print(f'Accuracy: {accuracy}')





# Sample Input:

sentence1 = "The cat is on the mat."

sentence2 = "The mat has a cat."



# Preprocess the sample input

sentence1_tokens = preprocess(sentence1)

sentence2_tokens = preprocess(sentence2)



# Check if entailment is predicted

is_entailment = simple_rule_based_entailment(sentence1_tokens, sentence2_tokens)

print(f'Entailment: {is_entailment}')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Accuracy: 0.33876719307182884
Entailment: False


B. Natural Language Inference with Bert

In [ ]:
# Step 1: Import Required Libraries

from datasets import load_dataset

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

import torch



# Step 2: Load the Dataset

dataset = load_dataset('snli')



# Check the first few examples to understand the structure

print(dataset['train'].features)  # Check the features of the training dataset

print(dataset['train'][0:5])       # Print the first 5 examples from the training dataset



# Step 3: Preprocess the Data

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



def preprocess_function(examples):

    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding='max_length', max_length=128)



# Apply preprocessing to the dataset (train, validation, and test splits)

encoded_dataset = dataset.map(preprocess_function, batched=True)



# Check the structure of the dataset again

print(encoded_dataset)



# Step 4: Inspect the label column directly to understand its structure

print("Label examples:")

print(encoded_dataset['train']['label'][0:5])  # Print the first 5 labels



# Step 5: Identify unique labels

unique_labels = set(encoded_dataset['train']['label'])

print(f"Unique labels in the dataset: {unique_labels}")



# Step 6: Define label mapping and handle unexpected labels

label_dict = {0: 0, 1: 1, 2: 2}  # Adjust this as necessary based on your labels



# Step 7: Map the labels correctly, handle unexpected labels

def map_labels(example):

    # Use the label_dict for mapping, and set a default for unexpected labels

    label = example['label']

    return {'labels': label_dict.get(label, -1)}  # Map to -1 if the label is unexpected



encoded_dataset = encoded_dataset.map(map_labels)



# Set the format for PyTorch

encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])



# Step 8: Load the Pre-Trained BERT Model

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)



# Step 9: Set Up Training Arguments and Trainer

training_args = TrainingArguments(

    output_dir='./results',          # Output directory

    evaluation_strategy='epoch',     # Evaluation during each epoch

    per_device_train_batch_size=16,  # Batch size for training

    per_device_eval_batch_size=16,   # Batch size for evaluation

    num_train_epochs=3,              # Number of training epochs

    weight_decay=0.01,               # Strength of L2 regularization

    logging_dir='./logs',            # Directory for logs

)



# Initialize the Trainer with the model, training arguments, and datasets

trainer = Trainer(

    model=model,                         # The BERT model for training

    args=training_args,                  # Training arguments

    train_dataset=encoded_dataset['train'],  # Training dataset

    eval_dataset=encoded_dataset['validation'],  # Validation dataset

)



# Step 10: Train the Model

trainer.train()



# Step 11: Evaluate the Model

eval_results = trainer.evaluate()

print(f"Evaluation Results: {eval_results}")



# Step 12: Make Predictions

premise = "A man inspects the uniform of a figure in some East Asian country."

hypothesis = "The man is sleeping."



# Tokenize the input example

inputs = tokenizer(premise, hypothesis, return_tensors='pt', padding=True, truncation=True, max_length=128)



# Get model prediction

model.eval()

with torch.no_grad():

    outputs = model(**inputs)

    predicted_label = torch.argmax(outputs.logits).item()



# Convert prediction to human-readable label

label_map = {0: 'entailment', 1: 'contradiction', 2: 'neutral'}

print(f"Predicted Label: {label_map[predicted_label]}")


{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}
{'premise': ['A person on a horse jumps over a broken down airplane.', 'A person on a horse jumps over a broken down airplane.', 'A person on a horse jumps over a broken down airplane.', 'Children smiling and waving at camera', 'Children smiling and waving at camera'], 'hypothesis': ['A person is training his horse for a competition.', 'A person is at a diner, ordering an omelette.', 'A person is outdoors, on a horse.', 'They are smiling at their parents', 'There are children present'], 'label': [1, 2, 0, 1, 0]}
DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_row

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/550152 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser 

In [ ]:
C) Sentence Pair Classification Using Siamese Networks

In [ ]:
# Required imports

from sentence_transformers import SentenceTransformer

from tensorflow.keras import layers, models, optimizers

import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

import numpy as np



# 1. Preprocess the Data: Load dataset and generate embeddings

data = pd.read_csv('sentence_similarity_dataset.csv')



# Load a pre-trained Sentence Transformer model for generating sentence embeddings

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')



# Get sentence embeddings

data['sentence1_embedding'] = data['sentence1'].apply(lambda x: model.encode(x))

data['sentence2_embedding'] = data['sentence2'].apply(lambda x: model.encode(x))



# Convert embeddings to numpy array

X1 = np.array(data['sentence1_embedding'].tolist())

X2 = np.array(data['sentence2_embedding'].tolist())

y = data['similarity_label'].values  # Assuming the dataset has a column for similarity labels



# Split the data into training and test sets

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)



# 2. Define the Siamese Network Architecture

# Input layers for the two sentences

input1 = layers.Input(shape=(X1.shape[1],))

input2 = layers.Input(shape=(X1.shape[1],))



# Dense layers for feature extraction

dense_layer = layers.Dense(128, activation='relu')

encoded1 = dense_layer(input1)

encoded2 = dense_layer(input2)



# Concatenate the extracted features and add a final dense layer for output

merged = layers.concatenate([encoded1, encoded2])

output = layers.Dense(1, activation='sigmoid')(merged)



# Define the Siamese model

siamese_model = models.Model(inputs=[input1, input2], outputs=output)



# Compile the model

siamese_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])



# 3. Train the model

siamese_model.fit([X1_train, X2_train], y_train, epochs=5, batch_size=32, validation_split=0.2)



# 4. Evaluate the Model

# Predict on the test set

y_pred = siamese_model.predict([X1_test, X2_test])



# Convert predictions to binary (similar or not similar)

y_pred = (y_pred > 0.5).astype(int)



# Calculate accuracy

accuracy = accuracy_score(y_test, y_pred)

print(f'Test Accuracy: {accuracy:.4f}')
